In [2]:
import numpy as np
import pickle
import glob
import matplotlib.pyplot as plt
import pandas as pd
from scipy import *
import os
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn import *
from sklearn.metrics import *
sns.set()
import warnings
warnings.filterwarnings("ignore")
length = 277

2024-03-06 18:17:49.148525: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-06 18:17:49.169786: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-06 18:17:49.169807: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-06 18:17:49.170317: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-06 18:17:49.174182: I tensorflow/core/platform/cpu_feature_guar

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 675930110860575368
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5803868160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10123267860980750116
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


2024-03-06 18:17:51.632365: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-06 18:17:51.662060: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-06 18:17:51.662096: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-06 18:17:51.894018: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-06 18:17:51.894061: I external/local_xla/xla/stream_executor

In [4]:
# Load the training and testing data:
train_values = np.empty(shape=[0, length])
test_values = np.empty(shape=[0, length])

train_beats = glob.glob('./data/train_test/train_patients.csv')
test_beats = glob.glob('./data/train_test/test_patients.csv')

for j in train_beats:
    print('Loading ', j)
    csvrows = np.loadtxt(j, delimiter=',')
    train_values = np.append(train_values, csvrows, axis=0)

for j in test_beats:
    print('Loading ', j)
    csvrows = np.loadtxt(j, delimiter=',')
    test_values = np.append(test_values, csvrows, axis=0)
    
print(train_values.shape)
print(test_values.shape)

# Separate the training and testing data, and one-hot encode Y:
X_train = train_values[:,:-2]
X_test = test_values[:,:-2]
y_train = train_values[:,-2]
y_test = test_values[:,-2]
X_test1 = X_test.reshape(-1, X_train.shape[1], 1)
y_test1=to_categorical(y_test)


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
X_train1 = X_train.reshape(-1, X_train.shape[1], 1)
y_train1=to_categorical(y_train)

X_val1 = X_val.reshape(-1, X_train.shape[1], 1)
y_val1=to_categorical(y_val)


Loading  ./data/train_test/train_patients.csv
Loading  ./data/train_test/test_patients.csv
(200352, 277)
(14482, 277)
X_train shape: (140246, 275)
X_val shape: (60106, 275)
y_train shape: (140246,)
y_val shape: (60106,)


In [5]:
from keras.layers import LSTM, Dropout, Dense,Flatten
from keras.optimizers import Nadam
from keras.losses import categorical_crossentropy  
from keras.utils import to_categorical
from keras import backend as K

def focal_loss(gamma=2.0):
    def focal_loss_fixed(y_true, y_pred):
        pt = y_pred * y_true + (1 - y_pred) * (1 - y_true)
        loss = -K.pow(1 - pt, gamma) * K.log(pt)
        return loss

    return focal_loss_fixed


# Initialize the model
model = Sequential()

model.add(LSTM(64, return_sequences=True, input_shape=(X_train1.shape[1],X_train1.shape[2])))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64, return_sequences=True))
model.add(Flatten())
# Add a Dense layer
model.add(Dense(9, activation='softmax'))  # Assuming 10 classes for classification

# Compile the model with focal loss
model.compile(loss=focal_loss(gamma=2.0), optimizer=Nadam(), metrics=['accuracy'])
model.summary()

2024-03-06 18:17:54.362174: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-06 18:17:54.362232: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-06 18:17:54.362247: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-06 18:17:54.362345: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-06 18:17:54.362360: I external/local_xla/xla/stream_executor

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 275, 64)           16896     
                                                                 
 lstm_1 (LSTM)               (None, 275, 64)           33024     
                                                                 
 lstm_2 (LSTM)               (None, 275, 64)           33024     
                                                                 
 lstm_3 (LSTM)               (None, 275, 64)           33024     
                                                                 
 flatten (Flatten)           (None, 17600)             0         
                                                                 
 dense (Dense)               (None, 9)                 158409    
                                                                 
Total params: 274377 (1.05 MB)
Trainable params: 274377 

In [6]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)


In [7]:
class_weights

array([0.99668827, 0.99873241, 1.00055648, 1.0023299 , 0.9971418 ,
       1.00324768, 1.00084209, 1.00049937])

In [8]:

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min',min_delta=0.001, restore_best_weights=True)
history = model.fit(X_train1, y_train1, validation_data=(X_val1, y_val1), epochs=50,callbacks=[early_stopping], verbose=1)




Epoch 1/50


2024-03-06 18:17:57.298978: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-06 18:17:57.653840: I external/local_xla/xla/service/service.cc:168] XLA service 0x7ff500326a00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-06 18:17:57.653878: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-03-06 18:17:57.659797: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1709749077.712950   70990 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4383/4383 [==============================] - 347s 78ms/step - loss: 0.0157 - accuracy: 0.9255 - val_loss: 0.0081 - val_accuracy: 0.9596
Epoch 2/50
4383/4383 [==============================] - 367s 84ms/step - loss: 0.0068 - accuracy: 0.9650 - val_loss: 0.0059 - val_accuracy: 0.9695
Epoch 3/50
4383/4383 [==============================] - 360s 82ms/step - loss: 0.0049 - accuracy: 0.9747 - val_loss: 0.0040 - val_accuracy: 0.9791
Epoch 4/50
4383/4383 [==============================] - 340s 78ms/step - loss: 0.0037 - accuracy: 0.9809 - val_loss: 0.0038 - val_accuracy: 0.9801
Epoch 5/50
4383/4383 [==============================] - 313s 71ms/step - loss: 0.0029 - accuracy: 0.9848 - val_loss: 0.0036 - val_accuracy: 0.9821
Epoch 6/50
4383/4383 [==============================] - 337s 77ms/step - loss: 0.0024 - accuracy: 0.9875 - val_loss: 0.0025 - val_accuracy: 0.9874
Epoch 7/50
4383/4383 [==============================] - 329s 75ms/step - loss: 0.0020 - accuracy: 0.9895 - val_loss: 0.0023 - val

In [9]:
class_counts = np.bincount(y_test.astype(int))
print(class_counts)


[   0 9363    0    0 1087    0  387 1138 2507]


In [10]:
from sklearn.metrics import confusion_matrix, f1_score

# Predict the test data
y_pred = model.predict(X_test1)

# Convert the predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Create the confusion matrix
cm = confusion_matrix(y_test, y_pred_labels)
print("Confusion Matrix:")
print(cm)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred_labels, average='weighted')
print("F1 Score:", f1)




453/453 [==============================] - 16s 33ms/step
Confusion Matrix:
[[9019    1    5   86   66   40  145    1]
 [   0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0]
 [   0    0    0 1081    0    6    0    0]
 [   0    0    0    0    0    0    0    0]
 [   1    0    0    0    0  386    0    0]
 [   0    0    0    0    0    0 1132    6]
 [   0    0    0    0    0    0   25 2482]]
F1 Score: 0.9764923609396889
